# San Francisco Crime Dataset Conversion

In [ ]:
import csv
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from StringIO import StringIO
from datetime import *
from dateutil.parser import parse

Initialize contexts and input file:

In [ ]:
sc = pyspark.SparkContext('local[*]')
sqlContext = SQLContext(sc)

In [ ]:
crime_df = sqlContext.read.parquet("data/sf-crime/*.parquet")

crime_df.printSchema()

In [ ]:
date_count = crime_df.groupBy('Dates', 'PdDistrict').count()

In [ ]:
#date_count.count()
date_count.sort('Dates', 'PdDistrict').show()

In [ ]:
# SQL fashion
crime_df.registerTempTable("crime")

sqlContext.sql("SELECT Category, count(*) as count FROM crime GROUP BY Category ORDER BY count DESC").show()